In [ ]:
# coding=utf-8
# Copyright 2023 Frank Latos AC8P
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#
# Much appreciation to the Pymoo project for providing the optimization framework used herein:
#
# pymoo: Multi-objective Optimization in Python
# https://github.com/anyoptimization/pymoo
# https://pymoo.org/index.html
#



Series section matching: Ludicrous Mode


In [48]:
import numpy as np
from numba import njit

import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import device_put
print("Device:", jax.devices()[0])


Device: gpu:0


In [49]:

# Precompute arrays A,B,C,D to speed impedance transformation calculations (lossless TL)
#   zoa     z of first matching section (connected to antenna)
#   zob     z of second matching section
#   flow, fhigh, nfreq      freq band of interest
#
#   Usage:  precompute A,B,C,D once for specified parameters
#       A,B,C,D = series_match_precompute(zoa=50,zob=75,nfreq=9,flow=3.5,fhigh=4.0)
#
#       z = (zs*A[a,b] + B[a,b]) / ((1j)*zs*C[a,b] + D[a,b])
#           where   zs is a row vector of complex zs, shape (1,nfreq)
#                   a,b   lengths of zoa,zob matching section (degrees)
#
@njit
def series_match_precompute(zoa=50,zob=75,nfreq=9,flow=3.5,fhigh=4.0):
    # Scale phase delay for freqs across band of interest
    phscale = (np.linspace(flow,fhigh,num=nfreq) / ((fhigh+flow)/2))[None,:]
    A = np.empty((181,181,nfreq))
    B = np.empty((181,181,nfreq), dtype=np.complex128)
    C = np.empty((181,181,nfreq))
    D = np.empty((181,181,nfreq))
    for a in range(181):
        tana = np.tan(np.deg2rad(a*phscale))
        for b in range(181):
            tanb = np.tan(np.deg2rad(b*phscale))
            A[a,b] = zoa*zob - zob**2 * tana * tanb
            B[a,b] = zoa*zob*(zoa*tana + zob*tanb)*(1j)
            C[a,b] = zob*tana + zoa*tanb
            D[a,b] = zoa*(zob - zoa * tana * tanb)
    return A,B,C,D

A,B,C,D = series_match_precompute(zoa=50,zob=75,nfreq=9,flow=3.5,fhigh=4.0)
Aj = device_put(A)
Bj = device_put(B)
Cj = device_put(C)
Dj = device_put(D)

In [51]:

# Generate 100 random feedpoint impedance curves (9 freqs)
zs = np.random.rand(100,9)*100 + (np.random.rand(100,9)*200 - 100)*(1j)


def transform_z(zr):
    return (zr*Aj + Bj) / ((1j)*zr*Cj + Dj)

zt = vmap(transform_z)(zs)


In [56]:

def find_min_vswr(zr, z0=50):
    zt = (zr*Aj + Bj) / ((1j)*zr*Cj + Dj)       # Transformed impedances for all segment length combinations
    refl_coef = jnp.abs((zt + z0) / (zt - z0))
    vswr = (1 - refl_coef) / (1 + refl_coef)
    max_vswr = jnp.max(vswr, axis=-1)

    idx = jnp.argmin(max_vswr)                           # Flat index
    return jnp.unravel_index(idx, max_vswr.shape)        

# zs = np.ones((100,9)) * (35-10j)
zs = np.random.rand(100,9)*100 + (np.random.rand(100,9)*200 - 100)*(1j)

vfunc = vmap(find_min_vswr)
seg_lens = vfunc(zs)
seg_lens


(Array([ 57, 118,   0, 134, 127, 146,  15, 134, 114,  74, 144, 152,  38,
        147,  45, 127,  49,  43, 101,  91, 115, 149,  10,  35, 117, 147,
        168,  30, 163,  35, 179, 154, 176, 153, 157, 167, 149,   7,  32,
         36, 101,  37, 156,  26,  25,  55, 103, 165,  28, 165, 169,  82,
         30, 157, 180,  22,  61, 160,  52, 155, 177,  89,  27,   1, 143,
         27,  11,   7, 124,  16, 128,  60, 138,  29,  33, 161, 131, 167,
         29,  34, 141, 156, 173,  49, 127, 177,   1,   0, 130,  19, 104,
         85,  30,  29,   9,  27, 154, 180,  90,  12], dtype=int32),
 Array([ 85, 148,  15,  91,  29,  88,  31,  85, 141,  26,  65, 133,  88,
         89, 139, 129,  40,  36, 110,  37,  32,  92, 123,  37,  38,  95,
        119, 169, 159,  61, 120,  92,  86,  66,  96,  98, 120,  47,  97,
        180,  15,  86,  58,  85,  45,  91, 177,  37,  85,  88,  46, 174,
        119, 166, 100, 103,  91,  96, 110, 134, 180, 174,  87, 112,  84,
         88, 158, 118, 139, 141,  85, 172, 100,  96,  88

In [57]:
%%timeit
seg_lens = vfunc(zs)


22.3 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
@njit
def series_match_scan(zs, A,B,C,D, step=1, z0=50.0):
    aopt, bopt, vswr_max_opt, vswr_curve_opt = (0, 0, 99999.0, None)
    for a in range(0,A.shape[0],step):
        for b in range(0,A.shape[1],step):

            z = (zs*A[a,b] + B[a,b]) / ((1j)*zs*C[a,b] + D[a,b])
            arf = np.abs((z - z0) / (z + z0))         # Reflection coefs
            vswr_curve = (1 + arf) / (1 - arf)
            vswr_max = np.max(vswr_curve)
            if vswr_max < vswr_max_opt:
                vswr_max_opt = vswr_max
                aopt = a
                bopt = b
                vswr_curve_opt = vswr_curve
    return aopt,bopt,vswr_curve_opt,vswr_max_opt

@njit
def series_match_array(zs):
    res = np.empty((zs.shape[0],2))
    for i in range(zs.shape[0]):
        a,b,_,_ = series_match_scan(zs[i:i+1], A,B,C,D)
        res[i,:] = (a,b)
    return res


In [62]:
%%timeit
series_match_array(zs)

2.32 s ± 8.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
%%timeit
series_match_scan(zs[0:1], A,B,C,D)

26.8 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [66]:
zs.shape

(100, 9)